<a href="https://colab.research.google.com/github/Egor1003/Data-Science/blob/main/%D0%A1%D0%B1%D0%BE%D1%80_%D0%B8_%D0%BE%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оглавление

# Сбор данных с сайта HeadHunter

Импорт необходимых библиотек

In [ ]:
import requests
import csv
import json
import time
import pandas as pd

Сбор вакансий по запросам из списка search с помощью API.

In [ ]:
def get_page(text, page = 0):
    params = {
        'text': text,
        'area': 113,
        'page': page,
        'per_page': 100,
    }

    req = requests.get('https://api.hh.ru/vacancies', params=params)
    data = req.content.decode()
    req.close()
    return data

js_objs = []
search = [
    'it',
    'программист',
    'frontend',
    'backend',
    'fullstuck',
    'аналитик',
    'ИТ',
    'разработчик',
    'developer',
    'дизайнер',
    'тестировщик',
    'информационные технологии'
]


for text in search:
    for page in range(0,20):
        js_obj = json.loads(get_page(text, page))

        js_objs.extend(js_obj['items'])

        if (js_obj['pages'] - page) <= 1:
            break

        # time.sleep(0.25)

print('Страницы поиска собраны')

In [ ]:
df = pd.DataFrame(js_objs)
df.to_csv('hh-ru.csv')
df.info()

In [ ]:
# df = pd.read_csv('hh-ru.csv')

# Очистка данных

Удаление дублирующихся вакансий в датасете по id.

In [ ]:
df1 = df.copy()
df1.drop_duplicates('id', inplace=True, ignore_index=True)
df1

Узнаем, в скольких вакансиях в датасете отсутствуют данные о зарплате и какие вылюты представлены в датасете

In [ ]:
d = df[df['salary'].notna()].salary.values
res = []
for item in d:
    currency = item.get('currency')
    if not currency in res:
        res.append(currency)

print(res)
print(len(d))

['RUR', 'USD', 'EUR']
12116


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         14729 non-null  object
 1   premium                    14729 non-null  bool  
 2   name                       14729 non-null  object
 3   department                 957 non-null    object
 4   has_test                   14729 non-null  bool  
 5   response_letter_required   14729 non-null  bool  
 6   area                       14729 non-null  object
 7   salary                     8468 non-null   object
 8   type                       14729 non-null  object
 9   address                    8398 non-null   object
 10  response_url               6 non-null      object
 11  sort_point_distance        0 non-null      object
 12  published_at               14729 non-null  object
 13  created_at                 14729 non-null  object
 14  archiv

Удалим ненужные столбцы

In [ ]:
columns = [
    'premium',
    'has_test',
    'relations',
    'accept_incomplete_resumes',
    'adv_response_url',
    'alternate_url',
    'adv_response_url',
    'apply_alternate_url',
    'url',
    'contacts',
    'insider_interview',
    'response_letter_required',
    'working_days',
    'working_time_intervals',
    'schedule',
    'sort_point_distance',
    'created_at', 'archived',
    'accept_temporary',
    'response_url'
    ]

df1.drop(columns=columns, inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  14729 non-null  object
 1   name                14729 non-null  object
 2   department          957 non-null    object
 3   area                14729 non-null  object
 4   salary              8468 non-null   object
 5   type                14729 non-null  object
 6   address             8398 non-null   object
 7   published_at        14729 non-null  object
 8   employer            14729 non-null  object
 9   snippet             14729 non-null  object
 10  working_time_modes  14729 non-null  object
 11  professional_roles  14729 non-null  object
 12  experience          14729 non-null  object
 13  employment          14729 non-null  object
dtypes: object(14)
memory usage: 1.6+ MB


In [ ]:
df1.drop(columns=['published_at'], inplace=True)

In [ ]:
df1.drop(columns=['department'], inplace=True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  14729 non-null  object
 1   name                14729 non-null  object
 2   area                14729 non-null  object
 3   salary              8468 non-null   object
 4   type                14729 non-null  object
 5   address             8398 non-null   object
 6   employer            14729 non-null  object
 7   snippet             14729 non-null  object
 8   working_time_modes  14729 non-null  object
 9   professional_roles  14729 non-null  object
 10  experience          14729 non-null  object
 11  employment          14729 non-null  object
dtypes: object(12)
memory usage: 1.3+ MB


Вытаскиваем из словарей данные о городах

In [ ]:
import numpy as np

def get_city(x):
    return x.get('name')

df1['area'] = df1['area'].apply(get_city)
df1.head()

,id,name,area,salary,type,address,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}",None,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}",None,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
2,81668501,Финансовый директор,Москва,None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': None, 'building':...","{'id': '4737', 'name': 'Razio Group', 'url': '...",{'requirement': 'Опыт работы в должности финан...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
3,81482773,"Диспетчер чатов, удаленно",Краснодар,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}",None,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
4,81662167,Front-end-разработчик,Москва,None,"{'id': 'open', 'name': 'Открытая'}",None,"{'id': '3562798', 'name': 'Гаскар Интеграция',...",{'requirement': 'Наличие опыта командной разра...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"


In [ ]:
df1.drop(columns=['address'], inplace=True)

Переводим данные о зарплатах в рубли типа float64

In [ ]:
def get_salary(x):
    try:
        if x['currency'] == 'RUR':
            return float(x['from'])
        elif x['currency'] == 'USD':
            return 77*float(x['from'])
        elif x['currency'] == 'EUR':
            return 84*float(x['from'])
    except TypeError:
        return np.NaN


df1['salary_clear'] = df1['salary'].apply(get_salary)
df1.head()

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment,salary_clear
0,81528929,"Диспетчер чатов, удаленно",Россия,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}","{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}","{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
2,81668501,Финансовый директор,Москва,None,"{'id': 'open', 'name': 'Открытая'}","{'id': '4737', 'name': 'Razio Group', 'url': '...",{'requirement': 'Опыт работы в должности финан...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",NaN
3,81482773,"Диспетчер чатов, удаленно",Краснодар,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}","{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
4,81662167,Front-end-разработчик,Москва,None,"{'id': 'open', 'name': 'Открытая'}","{'id': '3562798', 'name': 'Гаскар Интеграция',...",{'requirement': 'Наличие опыта командной разра...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",NaN


In [ ]:
df_type = pd.DataFrame(df1['type'])
df_type

,type
0,"{'id': 'open', 'name': 'Открытая'}"
1,"{'id': 'open', 'name': 'Открытая'}"
2,"{'id': 'open', 'name': 'Открытая'}"
3,"{'id': 'open', 'name': 'Открытая'}"
4,"{'id': 'open', 'name': 'Открытая'}"
...,...
14724,"{'id': 'open', 'name': 'Открытая'}"
14725,"{'id': 'open', 'name': 'Открытая'}"
14726,"{'id': 'open', 'name': 'Открытая'}"
14727,"{'id': 'open', 'name': 'Открытая'}"


In [ ]:
def get_name(x):
    return x['name']

df_type['type'] = df_type['type'].apply(get_name)

In [ ]:
df_type

,type
0,Открытая
1,Открытая
2,Открытая
3,Открытая
4,Открытая
...,...
14724,Открытая
14725,Открытая
14726,Открытая
14727,Открытая


In [ ]:
df_type['type'].unique()

array(['Открытая', 'Анонимная', 'Рекламная'], dtype=object)

In [ ]:
df1['type'] = df_type['type']
df1

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment,salary_clear
0,81528929,"Диспетчер чатов, удаленно",Россия,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...",Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...",Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
2,81668501,Финансовый директор,Москва,None,Открытая,"{'id': '4737', 'name': 'Razio Group', 'url': '...",{'requirement': 'Опыт работы в должности финан...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",NaN
3,81482773,"Диспетчер чатов, удаленно",Краснодар,"{'from': 26000, 'to': 40000, 'currency': 'RUR'...",Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",26000.0
4,81662167,Front-end-разработчик,Москва,None,Открытая,"{'id': '3562798', 'name': 'Гаскар Интеграция',...",{'requirement': 'Наличие опыта командной разра...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,"{'from': 90000, 'to': None, 'currency': 'RUR',...",Открытая,"{'id': '1268266', 'name': 'Машиностроительный ...",{'requirement': 'профильное или экономическое ...,[],"[{'id': '48', 'name': 'Инженер-конструктор, ин...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",90000.0
14725,81546295,Java Developer,Казань,None,Открытая,"{'id': '6093775', 'name': 'Aston', 'url': 'htt...",{'requirement': 'Опыт работы в роли Java Devel...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",NaN
14726,81666665,Системный аналитик,Волгоград,"{'from': 30000, 'to': None, 'currency': 'RUR',...",Открытая,"{'id': '2897406', 'name': 'ГБУ ВО ЦИТ ВО', 'ur...",{'requirement': '...сфере <highlighttext>инфор...,[],"[{'id': '148', 'name': 'Системный аналитик'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}",30000.0
14727,77875031,Стажер (ВТБ Юниор),Пенза,None,Открытая,"{'id': '4181', 'name': 'Банк ВТБ (ПАО)', 'url'...","{'requirement': 'Специальности: экономика, фин...",[],"[{'id': '70', 'name': 'Менеджер по продажам, м...","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'probation', 'name': 'Стажировка'}",NaN


In [ ]:
df1['salary'] = df1['salary_clear']
df1.drop(columns=['salary_clear'], inplace=True)
df1.head()

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
2,81668501,Финансовый директор,Москва,NaN,Открытая,"{'id': '4737', 'name': 'Razio Group', 'url': '...",{'requirement': 'Опыт работы в должности финан...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,Открытая,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
4,81662167,Front-end-разработчик,Москва,NaN,Открытая,"{'id': '3562798', 'name': 'Гаскар Интеграция',...",{'requirement': 'Наличие опыта командной разра...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"


In [ ]:
def get_employer(x):
    return x['name']

In [ ]:
df1['employer'] = df1['employer'].apply(get_employer)
df1

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,Открытая,Яндекс,{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,Открытая,Яндекс,{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
2,81668501,Финансовый директор,Москва,NaN,Открытая,Razio Group,{'requirement': 'Опыт работы в должности финан...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,Открытая,Яндекс,{'requirement': 'Опыт работы не требуется. Обя...,"[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
4,81662167,Front-end-разработчик,Москва,NaN,Открытая,Гаскар Интеграция,{'requirement': 'Наличие опыта командной разра...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"
...,...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,Открытая,Машиностроительный завод имени М.И.Калинина,{'requirement': 'профильное или экономическое ...,[],"[{'id': '48', 'name': 'Инженер-конструктор, ин...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"
14725,81546295,Java Developer,Казань,NaN,Открытая,Aston,{'requirement': 'Опыт работы в роли Java Devel...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
14726,81666665,Системный аналитик,Волгоград,30000.0,Открытая,ГБУ ВО ЦИТ ВО,{'requirement': '...сфере <highlighttext>инфор...,[],"[{'id': '148', 'name': 'Системный аналитик'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,Открытая,Банк ВТБ (ПАО),"{'requirement': 'Специальности: экономика, фин...",[],"[{'id': '70', 'name': 'Менеджер по продажам, м...","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'probation', 'name': 'Стажировка'}"


In [ ]:
def get_req(x):
    try:
        return x['requirement'].split('. ')
    except:
        return np.NaN

In [ ]:
df1['snippet'] = df1['snippet'].apply(get_req)

In [ ]:
df1

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
2,81668501,Финансовый директор,Москва,NaN,Открытая,Razio Group,[Опыт работы в должности финансового директора...,[],"[{'id': '135', 'name': 'Финансовый директор (C...","{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
4,81662167,Front-end-разработчик,Москва,NaN,Открытая,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"
...,...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,Открытая,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...",[],"[{'id': '48', 'name': 'Инженер-конструктор, ин...","{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}"
14725,81546295,Java Developer,Казань,NaN,Открытая,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...",[],"[{'id': '96', 'name': 'Программист, разработчи...","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}"
14726,81666665,Системный аналитик,Волгоград,30000.0,Открытая,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,[],"[{'id': '148', 'name': 'Системный аналитик'}]","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'full', 'name': 'Полная занятость'}"
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,Открытая,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...",[],"[{'id': '70', 'name': 'Менеджер по продажам, м...","{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'probation', 'name': 'Стажировка'}"


Фильтруем опыт работы, представляя его в int64

In [ ]:
import re
def get_experience(x):
    try:
        return int(re.sub('\D', ' ', x['name']).strip().split(' ')[0])
    except:
        return 0

df1['experience'] = df1['experience'].apply(get_experience)

Кодируем данные о занятости

In [ ]:
cleanup_nums = {'employment': {
                    'Полная занятость': 1,
                    'Частичная занятость': 2,
                    'Стажировка': 3,
                    'Проектная работа': 4
                }
            }

Очищаем данные о работодателях

In [ ]:
def get_empl(x):
    return x['name']

df1['employment'] = df1['employment'].apply(get_empl)
df1

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]",0,Полная занятость
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]",0,Полная занятость
2,81668501,Финансовый директор,Москва,NaN,Открытая,Razio Group,[Опыт работы в должности финансового директора...,[],"[{'id': '135', 'name': 'Финансовый директор (C...",6,Полная занятость
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...","[{'id': 'start_after_sixteen', 'name': 'Можно ...","[{'id': '40', 'name': 'Другое'}]",0,Полная занятость
4,81662167,Front-end-разработчик,Москва,NaN,Открытая,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,[],"[{'id': '96', 'name': 'Программист, разработчи...",1,Полная занятость
...,...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,Открытая,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...",[],"[{'id': '48', 'name': 'Инженер-конструктор, ин...",1,Полная занятость
14725,81546295,Java Developer,Казань,NaN,Открытая,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...",[],"[{'id': '96', 'name': 'Программист, разработчи...",3,Полная занятость
14726,81666665,Системный аналитик,Волгоград,30000.0,Открытая,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,[],"[{'id': '148', 'name': 'Системный аналитик'}]",0,Полная занятость
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,Открытая,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...",[],"[{'id': '70', 'name': 'Менеджер по продажам, м...",0,Стажировка


In [ ]:
df1 = df1.replace(cleanup_nums)

In [ ]:
def get_roles(x):
    return x[0]['name']

In [ ]:
df1['professional_roles'] = df1['professional_roles'].apply(get_roles)
df1['professional_roles']

0                                                   Другое
1                                                   Другое
2                                Финансовый директор (CFO)
3                                                   Другое
4                                 Программист, разработчик
                               ...                        
14724           Инженер-конструктор, инженер-проектировщик
14725                             Программист, разработчик
14726                                   Системный аналитик
14727    Менеджер по продажам, менеджер по работе с кли...
14728            Специалист по информационной безопасности
Name: professional_roles, Length: 14729, dtype: object

In [ ]:
df1['professional_roles'] = df1['professional_roles'].apply(lambda x: x.split(', '))

In [ ]:
def get_wtm(x):
    try:
        return x[0]['name']
    except:
        return np.NaN

In [ ]:
df1['working_time_modes'] = df1['working_time_modes'].apply(get_wtm)
df1

,id,name,area,salary,type,employer,snippet,working_time_modes,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",Можно начинать работать после 16:00,[Другое],0,1
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",Можно начинать работать после 16:00,[Другое],0,1
2,81668501,Финансовый директор,Москва,NaN,Открытая,Razio Group,[Опыт работы в должности финансового директора...,NaN,[Финансовый директор (CFO)],6,1
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,Открытая,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",Можно начинать работать после 16:00,[Другое],0,1
4,81662167,Front-end-разработчик,Москва,NaN,Открытая,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,NaN,"[Программист, разработчик]",1,1
...,...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,Открытая,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...",NaN,"[Инженер-конструктор, инженер-проектировщик]",1,1
14725,81546295,Java Developer,Казань,NaN,Открытая,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...",NaN,"[Программист, разработчик]",3,1
14726,81666665,Системный аналитик,Волгоград,30000.0,Открытая,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,NaN,[Системный аналитик],0,1
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,Открытая,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...",NaN,"[Менеджер по продажам, менеджер по работе с кл...",0,3


In [ ]:
df1.drop(columns=['working_time_modes'], inplace=True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  14729 non-null  object 
 1   name                14729 non-null  object 
 2   area                14729 non-null  object 
 3   salary              7621 non-null   float64
 4   type                14729 non-null  object 
 5   employer            14729 non-null  object 
 6   snippet             14559 non-null  object 
 7   professional_roles  14729 non-null  object 
 8   experience          14729 non-null  int64  
 9   employment          14729 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 1.1+ MB


In [ ]:
df1['type'].unique()

array(['Открытая', 'Анонимная', 'Рекламная'], dtype=object)

In [ ]:
columns_cleanup = {
    'type': {
        'Открытая': 1,
        'Анонимная': 2,
        'Рекламная': 3
    },
    'employment': {
        'Полная занятость': 1,
        'Частичная занятость': 2,
        'Стажировка': 3,
        'Проектная работа': 4
    },

}

In [ ]:
df1 = df1.replace(columns_cleanup)
df1

,id,name,area,salary,type,employer,snippet,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
2,81668501,Финансовый директор,Москва,NaN,1,Razio Group,[Опыт работы в должности финансового директора...,[Финансовый директор (CFO)],6,1
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
4,81662167,Front-end-разработчик,Москва,NaN,1,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,"[Программист, разработчик]",1,1
...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,1,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...","[Инженер-конструктор, инженер-проектировщик]",1,1
14725,81546295,Java Developer,Казань,NaN,1,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...","[Программист, разработчик]",3,1
14726,81666665,Системный аналитик,Волгоград,30000.0,1,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,[Системный аналитик],0,1
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,1,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...","[Менеджер по продажам, менеджер по работе с кл...",0,3


In [ ]:
df1['experience'] = df1['experience'].apply(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  14729 non-null  object 
 1   name                14729 non-null  object 
 2   area                14729 non-null  object 
 3   salary              7621 non-null   float64
 4   type                14729 non-null  int64  
 5   employer            14729 non-null  object 
 6   snippet             14559 non-null  object 
 7   professional_roles  14729 non-null  object 
 8   experience          14729 non-null  int64  
 9   employment          14729 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 1.1+ MB


In [ ]:

df1['employment'] = df1['employment'].apply(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14729 entries, 0 to 14728
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  14729 non-null  object 
 1   name                14729 non-null  object 
 2   area                14729 non-null  object 
 3   salary              7621 non-null   float64
 4   type                14729 non-null  int64  
 5   employer            14729 non-null  object 
 6   snippet             14559 non-null  object 
 7   professional_roles  14729 non-null  object 
 8   experience          14729 non-null  int64  
 9   employment          14729 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 1.1+ MB


In [ ]:
df1.to_csv('hh-ru.csv', index=False)

In [ ]:
df1[df1[['name','employer']].duplicated()].sort_values('name')

,id,name,area,salary,type,employer,snippet,professional_roles,experience,employment
1036,81554709,".NET Разработчик (Web, .Net Core)",Ярославль,NaN,1,it-aces.com,"[Обязательно: Опыт разработки Web решений, Опы...","[Программист, разработчик]",1,1
1383,81554765,".NET Разработчик (Web, .Net Core)",Новосибирск,NaN,1,it-aces.com,"[Обязательно: Опыт разработки Web решений, Опы...","[Программист, разработчик]",1,1
13230,78053748,.NET разработчик,Казань,NaN,1,Тинькофф,[Принимать и аргументировать собственные техни...,"[Программист, разработчик]",3,1
3263,80786702,.NET разработчик,Новосибирск,NaN,1,Диплей,"[Уверенные знания платформы .NET и языка C#, П...","[Программист, разработчик]",1,1
11910,80840647,3D-дизайнер,Санкт-Петербург,30000.0,1,Чесноков Сергей Анатольевич,"[Знание и опыт работы в 3D редакторах, Blender...","[Дизайнер, художник]",1,1
...,...,...,...,...,...,...,...,...,...,...
7202,81664635,Экономист-аналитик,Уфа,70000.0,1,Manpower,"[знание Excel на продвинутом уровне (ВПР, усло...",[Экономист],1,1
7095,81664636,Экономист-аналитик,Волгоград,70000.0,1,Manpower,"[знание Excel на продвинутом уровне (ВПР, усло...",[Экономист],1,1
6649,81356899,Экономист-стажер,Березники,NaN,1,УРАЛХИМ,[Аналитический склад ума и умение работать с б...,[Экономист],0,3
9968,81507805,Юрист-договорник / Юрисконсульт,Самара,60000.0,1,Соцков Роман Юрьевич,"[Высшее юридическое образование, - Опыт работы...",[Юрисконсульт],3,1


In [ ]:
df2 = df1.drop_duplicates(['name', 'employer', 'area'])
df2

,id,name,area,salary,type,employer,snippet,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
2,81668501,Финансовый директор,Москва,NaN,1,Razio Group,[Опыт работы в должности финансового директора...,[Финансовый директор (CFO)],6,1
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
4,81662167,Front-end-разработчик,Москва,NaN,1,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,"[Программист, разработчик]",1,1
...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,1,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...","[Инженер-конструктор, инженер-проектировщик]",1,1
14725,81546295,Java Developer,Казань,NaN,1,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...","[Программист, разработчик]",3,1
14726,81666665,Системный аналитик,Волгоград,30000.0,1,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,[Системный аналитик],0,1
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,1,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...","[Менеджер по продажам, менеджер по работе с кл...",0,3


In [ ]:
df2.to_csv('dataset.csv', index=False)

In [ ]:
df2[df2['salary'].notna()].to_csv('dataset.csv', index=False)

In [ ]:
df2

,id,name,area,salary,type,employer,snippet,professional_roles,experience,employment
0,81528929,"Диспетчер чатов, удаленно",Россия,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
1,81482770,"Диспетчер чатов, удаленно",Санкт-Петербург,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
2,81668501,Финансовый директор,Москва,NaN,1,Razio Group,[Опыт работы в должности финансового директора...,[Финансовый директор (CFO)],6,1
3,81482773,"Диспетчер чатов, удаленно",Краснодар,26000.0,1,Яндекс,"[Опыт работы не требуется, Обязательно наличие...",[Другое],0,1
4,81662167,Front-end-разработчик,Москва,NaN,1,Гаскар Интеграция,[Наличие опыта командной разработки Single Pag...,"[Программист, разработчик]",1,1
...,...,...,...,...,...,...,...,...,...,...
14724,81271950,Ведущий инженер по АСУП,Екатеринбург,90000.0,1,Машиностроительный завод имени М.И.Калинина,"[профильное или экономическое образование, - з...","[Инженер-конструктор, инженер-проектировщик]",1,1
14725,81546295,Java Developer,Казань,NaN,1,Aston,"[Опыт работы в роли Java Developer от 3-х лет,...","[Программист, разработчик]",3,1
14726,81666665,Системный аналитик,Волгоград,30000.0,1,ГБУ ВО ЦИТ ВО,[...сфере <highlighttext>информационных</highl...,[Системный аналитик],0,1
14727,77875031,Стажер (ВТБ Юниор),Пенза,NaN,1,Банк ВТБ (ПАО),"[Специальности: экономика, финансы, менеджмент...","[Менеджер по продажам, менеджер по работе с кл...",0,3


После очистки данных имеем 14620 вакансий

# Описание данных
* id --- уникальный номер вакансии
* name --- название вакансии
* area --- город
* salary --- предлагаемая заработная плата
* type --- тип вакансии
* employer --- работодетель
* snippet --- требуемые навыки
* professional_roles --- профессиональная деятельность
* experience --- опыт работы
* employment --- тип занятости
